<a href="https://colab.research.google.com/github/Bokdol11859/CNN-Pytorch/blob/main/AlexNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://proceedings.neurips.cc/paper/2012/file/c399862d3b9d6b76c8436e924a68c45b-Paper.pdf

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [ ]:
#Hyper-parameter
num_epochs = 20
batch_size = 40
learning_rate = 0.001
num_classes = 10

In [ ]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

train_dataset = torchvision.datasets.CIFAR10(root="./data", train=True, download=True, transform=transform)

test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform = transform)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size = batch_size, shuffle=True)

test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
class AlexNet(nn.Module):
  def __init__(self):
    super(AlexNet, self).__init__()
    self.conv1 = nn.Conv2d(in_channels=3, out_channels=96, kernel_size=11)
    self.relu = nn.ReLU()
    self.pool = nn.MaxPool2d(kernel_size=3, stride=2)
    self.conv2 = nn.Conv2d(in_channels=96, out_channels=256, kernel_size=5, padding=2)
    self.conv3 = nn.Conv2d(in_channels=256, out_channels=384, kernel_size=3, padding=1)
    self.conv4 = nn.Conv2d(in_channels=384, out_channels=384, kernel_size=3, padding=1)
    self.conv5 = nn.Conv2d(in_channels=384, out_channels=256, kernel_size=3, padding=1)

    self.avgpool = nn.AdaptiveAvgPool2d((6, 6))
    self.dropout = nn.Dropout(p=0.5)

    self.fc1 = nn.Linear(in_features=256 * 6 * 6, out_features=4096)
    self.fc2 = nn.Linear(in_features=4096, out_features=4096)
    self.fc3 = nn.Linear(in_features=4096, out_features=num_classes)

  def forward(self, x):
    #conv layer 1
    out = self.conv1(x)
    out = self.relu(out)
    out = self.pool(out)

    #conv layer 2
    out = self.conv2(out)
    out = self.relu(out)
    out = self.pool(out)

    #conv layer 3
    out = self.conv3(out)
    out = self.relu(out)

    #conv layer 4
    out = self.conv4(out)
    out = self.relu(out)

    #conv layer 5
    out = self.conv5(out)
    out = self.relu(out)
    out = self.pool(out)

    #average pool
    out = self.avgpool(out)

    # flatten
    out = out.view(-1, 256*6*6)

    #fully connected layer 1
    out = self.dropout(out)
    out = self.fc1(out)
    out = self.relu(out)

    #fully connected layer 2
    out = self.dropout(out)
    out = self.fc2(out)
    out = self.relu(out)

    #fully connected layer 3
    out = self.fc3(out)

    return out





In [ ]:
model = AlexNet().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
n_total_steps = len(train_loader)

for epoch in range(num_epochs):
  for i, (images, labels) in enumerate(train_loader):
    images = images.to(device)
    labels = labels.to(device)

    #Forward pass
    outputs = model(images)
    loss = criterion(outputs, labels)

    # Backward pass, update
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (i+1) % 10 == 0:
      print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')

print('Training Done')

Epoch [1/20], Step [10/1250], Loss: 2.3018
Epoch [1/20], Step [20/1250], Loss: 2.2995
Epoch [1/20], Step [30/1250], Loss: 2.2236
Epoch [1/20], Step [40/1250], Loss: 2.2325
Epoch [1/20], Step [50/1250], Loss: 2.3750
Epoch [1/20], Step [60/1250], Loss: 2.0171
Epoch [1/20], Step [70/1250], Loss: 2.0686
Epoch [1/20], Step [80/1250], Loss: 2.1885
Epoch [1/20], Step [90/1250], Loss: 2.0037
Epoch [1/20], Step [100/1250], Loss: 1.9177
Epoch [1/20], Step [110/1250], Loss: 2.0949
Epoch [1/20], Step [120/1250], Loss: 1.8458
Epoch [1/20], Step [130/1250], Loss: 2.1536
Epoch [1/20], Step [140/1250], Loss: 2.0175
Epoch [1/20], Step [150/1250], Loss: 2.0187
Epoch [1/20], Step [160/1250], Loss: 1.9939
Epoch [1/20], Step [170/1250], Loss: 2.0080
Epoch [1/20], Step [180/1250], Loss: 2.0106
Epoch [1/20], Step [190/1250], Loss: 1.9490
Epoch [1/20], Step [200/1250], Loss: 2.0735
Epoch [1/20], Step [210/1250], Loss: 1.9942
Epoch [1/20], Step [220/1250], Loss: 2.0768
Epoch [1/20], Step [230/1250], Loss: 2.04

In [ ]:
with torch.no_grad():
  n_correct = 0
  n_samples = 0
  n_class_correct = [0 for i in range(10)]
  n_class_samples = [0 for i in range(10)]
  for images, labels in test_loader:
    images = images.to(device)
    labels = labels.to(device)
    outputs = model(images)

    _, predicted = torch.max(outputs, 1)
    n_samples += labels.size(0)
    n_correct += (predicted == labels).sum().item()

    for i in range(batch_size):
      label = labels[i]
      pred = predicted[i]
      if (label == pred):
        n_class_correct[label] += 1
      n_class_samples[label] += 1

  acc = 100.0 * n_correct / n_samples
  print(f'Accuracy of the network: {acc}%')

  for i in range(10):
    acc = 100.0 * n_class_correct[i] / n_class_samples[i]
    print(f'Accuracy of {classes[i]}: {acc}%')

Accuracy of the network: 59.44%
Accuracy of plane: 74.8%
Accuracy of car: 69.6%
Accuracy of bird: 45.1%
Accuracy of cat: 26.6%
Accuracy of deer: 67.9%
Accuracy of dog: 41.3%
Accuracy of frog: 60.1%
Accuracy of horse: 68.4%
Accuracy of ship: 74.6%
Accuracy of truck: 66.0%
